# 1) Prodcue distance raster from point data

## 1.1 points inside 100m grid

In [1]:

## point to overlay with gridcell


import geopandas as gpd

# Load the polygon and point layers
polygons = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp")
points =   gpd.read_file(r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\sammelstellen\Altstoffsammelstellen_BIO_und_Mistplaetze_points_31256.shp')

output_grid_cell_with_points_inside = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.shp'


# Perform spatial join to find polygons covered by points
selected_polygons = gpd.sjoin(polygons, points, how="inner", op="contains")

# Display or manipulate the selected polygons as needed
print(selected_polygons.head())

# Save the selected polygons to a new shapefile if necessary
selected_polygons.to_file(output_grid_cell_with_points_inside , overwrite=True)

print ("done")


c:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_51260\103005694.py:20: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  selected_polygons.to_file(output_grid_cell_with_points_inside , overwrite=True)


        EAST     NORTH                         GRID100mID  \
4465 -9300.0  340500.0  EPSG31256_GRID_100m_E-9300N340500   
5622 -8800.0  341200.0  EPSG31256_GRID_100m_E-8800N341200   
7918 -7800.0  340800.0  EPSG31256_GRID_100m_E-7800N340800   
8841 -7400.0  341100.0  EPSG31256_GRID_100m_E-7400N341100   
8993 -7300.0  333300.0  EPSG31256_GRID_100m_E-7300N333300   

                                               geometry  index_right  BEZIRK  \
4465  POLYGON ((-9300.000 340500.000, -9300.000 3406...          488    14.0   
5622  POLYGON ((-8800.000 341200.000, -8800.000 3413...          454    14.0   
7918  POLYGON ((-7800.000 340800.000, -7800.000 3409...          906    14.0   
8841  POLYGON ((-7400.000 341100.000, -7400.000 3412...          445    14.0   
8993  POLYGON ((-7300.000 333300.000, -7300.000 3334...          891    23.0   

                   STRASSE      BEZUG   ONR   BIS  ...  \
4465        Mühlbergstraße  gegenüber    44  None  ...   
5622           Hauptstraße      nebe

# 1.2 raster from the resulting grids

In [7]:
##

## raster to vector using vector attriubte (working perfectly):

import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# Load the polygon layer
polygons = gpd.read_file(r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.shp')

output_raster = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.tif'


# Define the attribute to use for raster values
attribute_to_rasterize = 'raster_att'

# Define raster properties
xmin, ymin, xmax, ymax = polygons.total_bounds
pixel_size = 100  # Adjust as needed


width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_bounds(xmin, ymin, xmax, ymax, width, height)

# Create an empty array to hold rasterized values
raster_array = rasterize(
    [(geom, value) for geom, value in zip(polygons.geometry, polygons[attribute_to_rasterize])],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    all_touched=False,  # or True
    dtype=rasterio.uint8
)

# Write the raster to a GeoTIFF file
with rasterio.open(output_raster, "w", driver="GTiff", 
                   height=raster_array.shape[0], 
                   width=raster_array.shape[1], 
                   count=1, 
                   dtype=raster_array.dtype, 
                   crs=polygons.crs, 
                   transform=transform) as dst:
    dst.write(raster_array, 1)


print ("done")

done


# 2) Calculation the distance of every raster cell to the nearest raster cell with value (1 = location of biotonne)

In [10]:
## calc. 
# The gdal_proximity.py script generates a raster proximity map indicating the distance from the center of each pixel 
# to the center of the nearest pixel identified as a target pixel. 
# Target pixels are those in the source raster for which the raster pixel value is in the set of target pixel values.
# https://gdal.org/programs/gdal_proximity.html

import subprocess

# Full path to gdal_proximity.py
gdal_proximity_path = r"C:\ProgramData\anaconda3\Scripts\gdal_proximity.py"

# Input and output filenames
input_raster =  r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.tif'
output_raster =  r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m_distance_raster_1.tif'


# Command to run gdal_proximity.py
cmd = ["python", gdal_proximity_path, input_raster, output_raster]

# Run the command
subprocess.run(cmd)

print ("done")


CompletedProcess(args=['python', 'C:\\ProgramData\\anaconda3\\Scripts\\gdal_proximity.py', 'N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\Bio_sammel_mistplaetze_100m.tif', 'N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\Bio_sammel_mistplaetze_100m_distance_raster_1.tif'], returncode=0)